# Proyecto (obtención dataset)

In [2]:
# --- General ---
import os

# --- Data ---
import pandas as pd
# from pandasql import sqldf
# import numpy as np
# import statistics as stats

# --- Conexión ---
import elasticsearch

# --- Procesamiento lenguaje: spacy ---
import spacy
# from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher

# --- Procesamiento lenguaje: gensim ---
# import gensim
# import gensim.corpora as corpora
# from gensim.models import CoherenceModel

# --- Prediccion Positividad: POS, NEU, NEG ---
# from tqdm import tqdm
# import transformers
# from transformers import pipeline
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# --- Visualización ---
# import pyLDAvis
# import pyLDAvis.gensim_models 
# import matplotlib.colors as colors
# import matplotlib.pyplot as plt
# import geopandas as gpd
# %matplotlib inline

# ==============================================================================

# --- Configuración ---
# User password
password = os.environ.get('SOPHIA2')

# - - - geoData - - -
import tools_region as tr

# --- Funciones ---
# Cargar paquete de español mediano
nlp = spacy.load("es_core_news_md")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Utilización de regiones y comunas

In [3]:
data_regiones_comunas = tr.search_region('Región_de_Los_Lagos')
if(data_regiones_comunas):
    region = data_regiones_comunas['region']
    comunas_sql = data_regiones_comunas['comunas']
    comunas = [comuna.replace('_', ' ') for comuna in comunas_sql]

Region encontrada


## Inicialización peticiones Sophia2

In [3]:
# --- [Parametros de búsqueda] ---
# -- Nombre archivo --
nombre_archivo = 'data_Region_X' # Resultado: ./data/{nombre_archivo}_{from_}_{to_}.csv

# -- Parameters queries --
# País
country="chile"
# Fecha
from_="2022-01-01"
to_="2022-01-31"
# Medios de prensa - Region X
media_outlets=["elllanquihue","elaustral","laestrelladechiloe","elheraldoaustral","radiosago",
               "elrepuertero","elvacanudo","elhuemul","seminariolocal","elquellonino","elinsular",
               "radiopudeto","radioacogida","elcalbucano","segundos33",
               "prensadelestuario","fresiaahora","soychiloe"]

# Búsqueda de palabra clave
# keyword="pobreza"
# simple_keyword=True

In [4]:
# --- Data Connection ---
IP = "search.sophia2.org"
PORT = 9200
USER= "elastic"
PASS= password

# --- Connection ---
es = elasticsearch.Elasticsearch(
    IP,
    http_auth=(USER, PASS)
)

# --- Queries ---
query = { 
    "bool": { 
    "filter": [
        {"range": {
      "date": {
        "gte": from_,
        "lt": to_
      }
      }},

        { "term":  { "country": country }},
        { "terms":  { "media_outlet": media_outlets }} 
    ]
    }  
}

In [5]:
# --- Confirmación de conexión ---
res = es.search(index="news", query=query, size=10000)
print("Son %d noticias encontradas..." % res['hits']['total']['value'])

Son 1311 noticias encontradas...


## Creación dataframe

In [6]:
# --- Columnas Dataframe ---
data = {'id_news':[],'country':[],'media_outlet':[],'url':[],'title':[],'text':[],'date':[]}
df = pd.DataFrame(data)  

# --- Información Dataframe---
for hit in res['hits']['hits']:

    id_news = hit['_source']['id_news']
    country = hit['_source']['country']
    media_outlet = hit['_source']['media_outlet']
    url = hit['_source']['url']
    title = hit['_source']['title']
    text = hit['_source']['text']
    date = hit['_source']['date']
    
    new_row = {'id_news':int(id_news), 'country':country, 'media_outlet':media_outlet, 'url':url, 'title':title, 'text':text, 'date':date}
    df = pd.concat([df, pd.DataFrame.from_records([new_row])])
    # df = df.append(new_row, ignore_index=True)


# Reseteamos el índice del dataframe para que sea secuencial, sino serían puros 0.
df = df.reset_index(drop=True)

# --- Elimina resiuduos ---
# Elimina duplicados
df = df.drop_duplicates(subset='url', keep='first')
# Elimina textos vacíos o con muy poco contenido
df = df[df['text'].str.len() > 50]

In [7]:
print(f"Son {len(df)} noticias encontradas que se utilizarán")
df.head()

Son 1274 noticias encontradas que se utilizarán


,id_news,country,media_outlet,url,title,text,date
0,21907946.0,chile,radiosago,https://www.radiosago.cl/investigan-muerte-de-...,Investigan muerte de hombre apuñalado en Máfil...,La Fiscalía de Los Lagos dirige una investigac...,2022-01-26
1,21908053.0,chile,radiosago,https://www.radiosago.cl/ante-retroceso-a-fase...,Ante retroceso a Fase 3: Municipalidad de Puye...,Frente al alza de contagios de Covid-19 en la ...,2022-01-24
2,21908070.0,chile,radiosago,https://www.radiosago.cl/alcaldesa-de-puyehue-...,Alcaldesa de Puyehue dio positivo por covid-19,"La jefa comunal de la comuna lacustre, María J...",2022-01-23
3,21908071.0,chile,radiosago,https://www.radiosago.cl/el-primer-semestre-de...,El primer semestre de este año entrará en func...,La jefa de la División de Atención Primaria de...,2022-01-23
4,21908072.0,chile,radiosago,https://www.radiosago.cl/organizaciones-de-la-...,Organizaciones de la región de Los Lagos crean...,Como una necesidad por plantear la urgencia qu...,2022-01-24


## Añadiremos las comunas mencionadas en las noticias

In [8]:
# --- Creamos Matcher utilizando nlp ( natural language processing ) ---
matcher_comunas = PhraseMatcher(nlp.vocab)
for comuna in comunas:
    matcher_comunas.add(comuna, [nlp(comuna)])

# --- Función para buscar comunas ---
for index,row in df.iterrows():
    txt = row["text"]
    
    try:
        # -- Procesamos el texto --
        doc = nlp(txt)
        matches_comunas = matcher_comunas(doc)

        # -- Creamos lista de comunas --
        for match_id, start, end in matches_comunas:
            span = doc[start:end]  # The matched span

    except:
        pass

In [9]:
# --- Reemplazamos los espacios por guiones bajos, para cada Comuna ---
# ejemplo: "San Juan de la Costa" -> "San_Juan_de_la_Costa"
df_comunas = df
for comuna in comunas:
    df_comunas.insert(7,comuna.replace(" ","_"),0)

# --- Insertamos las comunas al dataframe (df) ---
for index,row in df_comunas.iterrows():
    txt = row["text"]
    
    try:
    
        doc = nlp(txt)
        matches_comunas = matcher_comunas(doc)

        for match_id, start, end in matches_comunas:
            span = doc[start:end]  # The matched span
            df_comunas.at[index,span.text.replace(" ","_")]=1
    except Exception as e:
        print(e)
        pass

## Guardamos la información CSV

In [10]:
# --- Info Archivo ---
nombre_archivo= nombre_archivo+"_"+from_+"_"+to_+".csv"
print("El archivo se llama:")
print(nombre_archivo)

# --- Guarda archivo ---
df_comunas.to_csv("./data/"nombre_archivo)

El archivo se llama:
data_Region_X_2022-01-01_2022-01-31.csv


###  Con esto tenemos toda la información necesaria para cargar los datos y utilizarlos.